# CHSolver User Tutorial

## Requirements
We assume the following software will be already available prior to installation:
- GCC/10.2.0
- CMake/3.20.1
- OpenMPI/4.0.5
- HDF5/1.10.7
- Python/3.8.6
- FFTW/3.3.8

## Installation
The repository can be git cloned via the command  $\texttt{ git clone --recursive git@github.com:HetSys/CHSolver.git}$

The code requires some external dependancies, which are linked to the repository through the use of git submodules. The bash script $\texttt{build\_deps}$ is provided in order to build and install these locally, within the repository.

Make is used as the main driver for the executable compilation and support. Supported commands are as follows:
- make: Compile and link the chsolver executable
- make clean: Clean the executables, .o and .mod files
- make logpurge: Clean all .log files from the ./logs dir
- make docs: Generate developer documentation (requires a doxygen executable on path, tested with Doxygen/1.8.17)
- make tests: Compile and link a unit tests executable (requires pFUnit to have been installed within the repository)

## Handling JSON input data
JSON files are used by both the Python API and the $\texttt{chsolver}$ executable in order to more easily handle input parameters to the executable. Though it is possible to use the the executable without requiring a JSON file, it is strongly recommended.

The JSON datastructure operates via nesting of key : value pairs. At the root level, the JSON file is subdivided into "runs", where each run contains the full set of parameters required for proper execution of the program. A simple example file is shown below.

```
{
  "default": {
    "L": 1.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -1.0,
    "p1": 1.0,
    "grid_level": 7,
    "grid_type": "r",
    "T": [
      0.0,
      0.05,
      0.1
    ]
  }
}
```

The run, called "default" contains a nested set of parameters ("L", "A", "M", ...) which would all be searched by name during execution of the program. To add alternative runs directly to the file, it is recommended to first copy and paste the default run, rename it, and thn modify the values associated with the parameters.

### Handling the JSON file from Python
The CHData class (from src/dataclass.py) provides an interface to generate and modify JSON files interactively.
A worked example of generating and adding a new run to a json file from the Python API is shown below. 

In [16]:
# IMPORTS
import numpy as np
import os
from src.dataclass import CHData


In [ ]:

# Generating an instance of CHData will create or open the file given by the fname init arg.
# If the file did not already exist, the default run will be saved to file
dat = CHData(fname="tutorial-data.json")

# Print the input data currently loaded
# By default, this will be the data contained in the "default" run in the
dat.print_rundata()


In [8]:
# Modify the input data stored in the CHData object
dat.L = 2.0

dat.p0 = -3.0

dat.grid_level = 4

dat.T = np.linspace(0, 0.2, 5)

dat.grid_type = "c"

# Print out the modifications made to the input data
dat.print_rundata()

L = 2.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -3.0
p1 = 1.0
Grid type = c
Grid level = 4
Output Times = [0.   0.05 0.1  0.15 0.2 ]


In [9]:
# Save the input data back down to the file
dat.save_rundata(run_name="modified")

Opening the "tutorial-data.json" file, it should now look like this:
```
{
  "default": {
    "L": 1.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -1.0,
    "p1": 1.0,
    "grid_level": 7,
    "grid_type": "r",
    "T": [
      0.0,
      0.05,
      0.1
    ]
  },
  "modified": {
    "L": 2.0,
    "A": 1.0,
    "M": 0.25,
    "K": 0.0004,
    "p0": -3.0,
    "p1": 1.0,
    "grid_level": 4,
    "grid_type": "c",
    "T": [
      0.0,
      0.05,
      0.1,
      0.15000000000000002,
      0.2
    ]
  }
}
```

Now we have a file containing multiple runs, we can look into quickly swapping between them.


In [11]:
# Create a new object, opening the same file
dat2 = CHData(fname="tutorial-data.json")

# Print the run names found in the file
print(dat2.run_names)


['default', 'modified']
L = 1.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -1.0
p1 = 1.0
Grid type = r
Grid level = 7
Output Times = [0.   0.05 0.1 ]


In [13]:
# Print the currently loaded run data
dat2.print_rundata()

L = 2.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -3.0
p1 = 1.0
Grid type = c
Grid level = 4
Output Times = [0.0, 0.05, 0.1, 0.15000000000000002, 0.2]


In [12]:
# Read the data contained in the "modified" datastructure
dat2.read_rundata("modified")

# 
dat2.print_rundata()

L = 2.0
A = 1.0
M = 0.25
K = 0.0004
p0 = -3.0
p1 = 1.0
Grid type = c
Grid level = 4
Output Times = [0.0, 0.05, 0.1, 0.15000000000000002, 0.2]


## Using the $\texttt{chsolver}$ executable

The $\texttt{chsolver}$ can be built via the Makefile, using the command $\texttt{make}$. The executable comes with a fully featured Command Line Interface in order to allow the program to be flexible to a variety of tasks.

To demonstrate the program for this tutorial, we run shell commands via the python os.system() function.

In [15]:
# OPTIONAL: Compile the chsolver executable (if not already build)
os.system("make clean && make")

rm -f -r -d chsolver  ./obj/*.o ./src/*.mod ./out/** ./doxygen/output/* test_all
make -C tests/unit_tests clean
make[1]: Entering directory '/home/u2180064/Documents/GitHub/CHSolver/tests/unit_tests'
rm -f *.o *.mod *.a  *.inc *.F90
make[1]: Leaving directory '/home/u2180064/Documents/GitHub/CHSolver/tests/unit_tests'
mkdir -p ./obj ./out ./logs
Building obj/face.o
h5fc  -std=f2008 -c -o obj/face.o src/submodules/FACE/src/lib/face.F90 
Building obj/logger_mod.o
h5fc  -std=f2008 -c -o obj/logger_mod.o src/submodules/flogging/src/logging.f90 
Building obj/logging.o
h5fc  -J./src -I./src/submodules/bin/jsonfortran-gnu-8.2.5/lib -Wall -Wextra -Wconversion-extra -std=f2008 -g -fall-intrinsics -c -o obj/logging.o src/logging.f90 
Building obj/globals.o
h5fc  -J./src -I./src/submodules/bin/jsonfortran-gnu-8.2.5/lib -Wall -Wextra -Wconversion-extra -std=f2008 -g -fall-intrinsics -c -o obj/globals.o src/globals.f90 
Building obj/command-line.o
h5fc  -J./src -I./src/submodules/bin/jsonfortran-gn

0

### Basic Usage

In [20]:
# Check the version of the program
version_command = "./chsolver --version"
os.system(version_command)

# Print help text

help_command = "./chsolver -h"
#os.system(help_command)


 CHSolver 1.0


In [22]:
# IO Commands

json_file = "tutorial-data.json"

output_dir = "tutorial-out"
os.makedirs(output_dir, exist_ok=True)

run_name = "modified"

exe = "./chsolver"

# -j <file> will change the json file searched for inputs
json_file_command = f" -j {json_file}"

# -o <dir> will change the location output files are saved to
output_dir_command = f" -o {output_dir}"

# -r <name> will change the name of the run parsed in the JSON for inputs 
run_name_command = f" -r {run_name}"


# Generate the full command
basic_command = exe + json_file_command + output_dir_command + run_name_command

print(basic_command)
os.system(basic_command)

./chsolver -j tutorial-data.json -o tutorial-out -r modified
[May 24 2022 13:23:49][open_json] <info> Reading tutorial-data.json


At line 272 of file src/hdf5-io.f90
Fortran runtime error: Cannot open file 'tutorial-out/metadata.dat': No such file or directory

Error termination. Backtrace:
#0  0x7f7a539c0d21 in ???
#1  0x7f7a539c1869 in ???
#2  0x7f7a539c254f in ???
#3  0x7f7a53c019d6 in ???
#4  0x7f7a53c01ccc in ???
#5  0x55c93ce7236c in __hdf5_io_MOD_chkpnt_init
	at src/hdf5-io.f90:272
#6  0x55c93ce710de in MAIN__
	at src/main.f90:101
#7  0x55c93ce71a85 in main
	at src/main.f90:2


512

# Output Files

## Working with output files from Python

The CHData class also provides functionality for working with the output files produced by the $\texttt{chsolver}$ program.

In [ ]:
dat = CHData(fname="tutorial-data.json")

